In [3]:
import wave
import json
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment

def convert_to_wav(audio_path, output_path="converted.wav"):
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export(output_path, format="wav")
    return output_path

def transcribe_audio(audio_path, model_path="model"):
    wf = wave.open(audio_path, "rb")
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
        raise ValueError("Audio must be WAV format mono PCM with 16kHz sample rate")

    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())

    text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            text += result.get("text", "") + " "
    result = json.loads(rec.FinalResult())
    text += result.get("text", "")
    return text.strip()

# Example usage
input_audio = "./Deloitte_interview_1.m4a"  # Or .wav, .flac etc.
wav_file = convert_to_wav(input_audio)
text = transcribe_audio(wav_file)
print("Extracted Text:\n", text)


FileNotFoundError: [WinError 2] The system cannot find the file specified